In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 681.0 kB/s eta 0:00:00


In [37]:
# Import necessary library
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch import nn
import torch.nn.functional as F
from einops import rearrange,repeat
from einops.layers.torch import Rearrange
import math
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler

In [38]:
# Reimplement Autoformer's classes to build model

# Normalize layers based on number of features
class LayerNorm(nn.Module):
    def __init__(self,nfeat):
        super().__init__()
        self.norm = nn.LayerNorm(nfeat)

    def forward(self,X):
        X_norm = self.norm(X)
        return X_norm - X_norm.mean(dim = 1).unsqueeze(dim = 1)

# Moving average
class MovingAvg(nn.Module):
    def __init__(self, kernel_size, stride):
        super().__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # Padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

# Decomposite the time series
class SeriesDecomp(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.moving_avg = MovingAvg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

# Auto attention instead of Full attention in Transformer
class AutoAttentionLayer(nn.Module):
    def __init__(self,nfeat,nhid,nhead, kernel_size = 25, dropout = 0.05):
        super().__init__()
        self.q = nn.Linear(nfeat,nhid*nhead)
        self.k = nn.Linear(nfeat,nhid*nhead)
        self.v = nn.Linear(nfeat,nhid*nhead)
        self.nhead = nhead
        self.out = nn.Linear(nhead*nhid,nfeat)

        self.decomp1 = SeriesDecomp(kernel_size)
        self.decomp2 = SeriesDecomp(kernel_size)
        self.dropout = nn.Dropout(dropout)

        self.ffn = nn.Sequential(
            Rearrange('b l f -> b f l'),
            nn.Conv1d(nfeat,2*nfeat,kernel_size = 3, padding=1, bias = False),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Conv1d(2*nfeat,nfeat,kernel_size = 3, padding=1, bias = False),
            nn.Dropout(dropout),
            Rearrange('b f l -> b l f')
        )

    def forward(self,X,Y=None,Z=None,return_trend = False):
        self.topk = int(math.log(X.shape[1])+1)

        if Y is None and Z is None:
            qkv = (self.q(X),self.k(X),self.k(X))
        else:
            qkv = (self.q(X),self.k(Y),self.v(Z))

        q,k,v = map(lambda x: rearrange(x,'b t (h f)-> b h f t',h = self.nhead),qkv)

        Tq,Tk = q.shape[-1],k.shape[-1]
        if Tq > Tk:
            v = F.pad(v,(0,Tq-Tk),"constant",0)
            k = F.pad(k,(0,Tq-Tk),"constant",0)
        else:
            v = v[:,:Tq]
            k = k[:,:Tq]

        # Get time series autocorrelation
        q_fft = torch.fft.rfft(q,dim=-1)
        k_fft = torch.conj(torch.fft.rfft(k,dim=-1))
        lag = torch.fft.irfft(q_fft*k_fft)
        # Use lag
        corr = lag.mean(dim=(1,2))
        weights,delays = torch.topk(corr,self.topk,dim=-1)
        delays = torch.topk(weights.mean(dim=0),self.topk,dim=-1)[1]
        weights = torch.stack([weights[:,idx] for idx in delays],dim = -1)
        weights  = F.softmax(weights,dim=-1)
        # Fusion
        values = v.clone()
        for idx in range(self.topk):
            values[:,:,:,idx] = torch.roll(v[:,:,:,idx],-int(delays[idx]),-1) * rearrange(weights[:,idx],"b ->b 1 1 ")

        values = self.out(rearrange(values,'b h f t -> b t (h f)'))

        X = X + self.dropout(values)

        # Get residual (seasonality)
        y, trend1 = self.decomp1(X)
        y = self.ffn(y) #ffn
        res,trend2 = self.decomp2(X+y)

        if not return_trend:
            return res
        else:
            return res,trend1+trend2

# Encoder to encode time series input
class Encoder(nn.Module):
    def __init__(self,nlayers = 2,nfeat = 2048,nhid = 256,nhead = 8, kernel_size = 25,dropout = 0.05):
        super().__init__()
        self.modulelist = nn.ModuleList([AutoAttentionLayer(nfeat,nhid,nhead,kernel_size,dropout) for i in range(nlayers)])
        self.norm = LayerNorm(nfeat)

    def forward(self,X):
        for m in self.modulelist:
            X = m(X)
        X = self.norm(X)
        return X

# Decoder layer to decode and give prediction
class DecoderLayer(nn.Module):
    def __init__(self,nfeat, nembed = 2048,nhid = 256,nhead = 8, kernel_size = 25,dropout = 0.05):
        super().__init__()
        self.self_attn = AutoAttentionLayer(nembed,nhid,nhead,kernel_size,dropout)
        self.cross_attn = AutoAttentionLayer(nembed,nhid,nhead,kernel_size,dropout)
        self.dropout = nn.Dropout(dropout)
        self.decomp = SeriesDecomp(kernel_size)
        self.projection = nn.Sequential(
                Rearrange('b t f-> b f t'),
                nn.Conv1d(in_channels=nembed, out_channels=nfeat, kernel_size=3, padding=1,
                                        padding_mode='circular', bias=False),
                Rearrange('b f t -> b t f'),
            )

    def forward(self,X,Y):
        X = X + self.dropout(self.self_attn(X))
        season1,trend1 = self.decomp(X)

        seanson2, trend2 = self.cross_attn(season1,Y,Y,return_trend = True)
        season = season1 + self.dropout(seanson2)

        trend = self.projection(trend1+trend2)
        # return X,trend
        return season,trend

class Decoder(nn.Module):
    def __init__(self,nlayers = 1,nfeat = 8 , nembed = 2048,nhid = 256,nhead = 8, kernel_size = 25,dropout = 0.05):
        super().__init__()
        self.modulelist = nn.ModuleList([DecoderLayer(nfeat,nembed,nhid,nhead,kernel_size,dropout) for i in range(nlayers)])
        self.norm = LayerNorm(nfeat)
        self.projection = nn.Sequential(
                Rearrange('b t f-> b f t'),
                nn.Conv1d(in_channels=nembed, out_channels=nfeat, kernel_size=3, padding=1,
                                        padding_mode='circular', bias=False),
                Rearrange('b f t -> b t f'),
            )

    def forward(self,X,Y,trend):
        for m in self.modulelist:
            X,residual_trend = m(X,Y)
            trend += residual_trend

        X = self.norm(self.projection(X))
        return X,trend

# Use position embedding to avoid no time stamp input
class PosEmbedding(nn.Module):
    def __init__(self,nfeat,max_length = 20000):
        super().__init__()
        position = torch.arange(0, max_length).float().unsqueeze(1)
        div_term = (torch.arange(0, nfeat, 2).float() * -(math.log(10000.0) / nfeat)).exp()

        self.pe = torch.zeros(max_length, nfeat).float()
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0)
    def forward(self, x):
        return self.pe[:, :x.size(1),:].to(x.device)

# Temporal embedding can highly improve the performance
class TemporalEmbedding(nn.Module):
    def __init__(self, nfeat, max_size = 100, max_stamps = 20):
        super(TemporalEmbedding, self).__init__()
        self.max_stamps = max_stamps
        self.max_size = max_size

        self.embed_list = nn.ModuleList()
        for i in range(max_stamps):
            self.embed_list.append(nn.Embedding(max_size, nfeat))

    def forward(self, x):
        num_stamps = x.shape[-1]
        x = x.long()

        embeddings_list = []
        for i in range(num_stamps):
            embed = self.embed_list[i]
            embeddings_list.append(embed(x[:,:,i]))
        return sum(embeddings_list)

# Autoformer model class
class AutoFormer(nn.Module):
    def __init__(self,pred_length = None ,encoder_layers=2,decoder_layers=1,nfeat = 8,nembed = 2048,nhid = 256,nhead = 8, kernel_size = 25,dropout = 0.05):
        super().__init__()
        self.enc_embedding = nn.Sequential(
                Rearrange('b t f-> b f t'),
                nn.Conv1d(in_channels=nfeat, out_channels=nembed, kernel_size=3, padding=1,
                                        padding_mode='circular', bias=False),
                Rearrange('b f t -> b t f'),
            )
        self.dec_embedding = nn.Sequential(
                Rearrange('b t f-> b f t'),
                nn.Conv1d(in_channels=nfeat, out_channels=nembed, kernel_size=3, padding=1,
                                        padding_mode='circular', bias=False),
                Rearrange('b f t -> b t f'),
            )
        self.trend_projection = nn.Linear(nembed,nfeat)
        self.seasonal_projection = nn.Linear(nembed,nfeat)

        self.encoder = Encoder(encoder_layers,nembed,nhid,nhead,kernel_size,dropout)
        self.decoder = Decoder(decoder_layers,nfeat,nembed,nhid,nhead,kernel_size,dropout)

        self.decomp = SeriesDecomp(kernel_size)

        self.pos_embedding = PosEmbedding(nembed)
        self.time_embedding = TemporalEmbedding(nembed)
        self.pred_length = pred_length

    def forward(self,X_enc,X_stamps = None,y_stamps = None):
        T = X_enc.shape[1]
        # Decoder input preparation
        mean = torch.mean(X_enc, dim=1).unsqueeze(1).repeat(1, self.pred_length, 1)
        zeros = torch.zeros_like(mean,device = X_enc.device)

        # Decomposition
        seasonal_init,trend_init = self.decomp(X_enc)
        trend_init = torch.cat([trend_init[:, -self.pred_length:, :], mean], dim=1)
        seasonal_init = torch.cat([seasonal_init[:, -self.pred_length:, :], zeros], dim=1)

        # Encoder
        X_enc = self.enc_embedding(X_enc)
        if X_stamps is None:
            X_enc += self.pos_embedding(X_enc)
        else:
            X_enc += self.time_embedding(X_stamps)

        enc_out = self.encoder(X_enc)
        # Decoder
        X_dec = self.dec_embedding(seasonal_init)
        if y_stamps is None:
            X_dec += self.pos_embedding(X_dec)
        else:
            X_dec[:,-self.pred_length:,:] = X_dec[:,-self.pred_length:,:] + self.time_embedding(y_stamps)

        seasonal_part,trend_part = self.decoder(X_dec,enc_out,trend_init)
        X = seasonal_part + trend_part
        X = X[:,-self.pred_length:,:]

        return X

In [39]:
# Class to read and process dataset, split parameter is used to split to train and test set
class dataset(Dataset):
    def __init__(self,input_length = 96,preds_length = 96 ,path = '/content/drive/MyDrive/Third Year/Sem 2/Business Data Analyst/dataset/Gold.csv',
                 train = 'train',split = (0.7,0.3)):
        super().__init__()
        df = pd.read_csv(path)
        # Time series feature
        df_stamp = pd.DataFrame()
        df['Date'] = pd.to_datetime(df['Date'])
        df_stamp['Month'] = df['Date'].apply(lambda row:row.month,1)
        df_stamp['Weekday'] = df['Date'].apply(lambda row:row.weekday(),1)
        df_stamp['Hour'] = df['Date'].apply(lambda row:row.hour,1)
        df_stamp['Day'] = df['Date'].apply(lambda row:row.day,1)
        df_stamp = df_stamp[['Month','Weekday','Hour','Day']].values

        # Scaled data
        cols = ['Open','Close','High','Low']
        df = df[cols]
        self.scale_fn = StandardScaler()
        train_length = int((len(df)-input_length-preds_length)*split[0])
        test_length = int((len(df)-input_length-preds_length)*split[1])
        train_data = df[:train_length].values
        self.scale_fn.fit(train_data)
        df = self.scale_fn.transform(df.values)

        if train == 'train':
            self.df = df[:train_length]
            self.df_stamp = df_stamp[:train_length]
        else:
            self.df = df[train_length+test_length:]
            self.df_stamp = df_stamp[train_length+test_length:]

        self.input_length = input_length
        self.preds_length = preds_length
        self.train = train

        self.df = torch.tensor(self.df,device = 'cuda' if torch.cuda.is_available() else 'cpu').float()
        self.df_stamp = torch.tensor(self.df_stamp,device = 'cuda' if torch.cuda.is_available() else 'cpu')


    def __inverse_transform__(self,X):
        return self.scale_fn.inverse_transform(X)

    def __len__(self):
        return len(self.df) - self.input_length - self.preds_length + 1

    def __getitem__(self,index):
        X = self.df[index:index+self.input_length]
        X_stamp = self.df_stamp[index:index+self.input_length]

        y = self.df[index+self.input_length:index+self.input_length+self.preds_length]
        y_stamp = self.df_stamp[index+self.input_length:index+self.input_length+self.preds_length]

        return X,X_stamp,y,y_stamp

In [40]:
# Function used to train model
def train_per_epoch(dataloader):
    model.train()
    loss_report = 0.
    bar = tqdm(dataloader)
    for X,X_stamp,y,y_stamp in bar:
        optim.zero_grad()
        X = X.float()
        y = y.float()
        pred = model(X,X_stamp,y_stamp)
        loss = loss_fn(y,pred)
        loss.backward()
        optim.step()
        schedule.step()
        loss_report += loss.clone().detach().cpu().item()
        # bar.set_description("Train loss {:.4f} ".format(loss.clone().detach().cpu().item()))
    return loss_report/len(dataloader)

# Evaluate model after training
def valid_per_epoch(dataloader):
    model.eval()
    loss_report = 0.
    bar = tqdm(dataloader)
    preds = []
    labels = []
    for X,X_stamp,y,y_stamp in bar:
        with torch.no_grad():
            X = X.float()
            y = y.float()
            pred = model(X,X_stamp,y_stamp)
            loss = loss_fn(y,pred)
            loss_report += loss.clone().detach().cpu().item()
            # bar.set_description("Valid loss {:.4f}".format(loss.clone().detach().cpu().item()))

            preds.append(pred.clone().detach().cpu().numpy())
            labels.append(y.clone().detach().cpu().numpy())

    preds = np.concatenate(preds,axis = 0)
    labels = np.concatenate(labels,axis = 0)
    loss_report = {
        'loss': loss_report/len(dataloader),
        'mape': np.abs(((labels-preds)/labels)).mean(),
        'mse': ((labels-preds)**2).mean(),
        'rmse': math.sqrt(((labels-preds)**2).mean())
    }
    return loss_report

def predict_future(dataloader, device):
    model.eval()
    y_pred_list = []
    y_list = []
    with torch.no_grad():
        for index,samples in enumerate(dataloader):
            x,y_label=samples
            x = x.to(device)
            y_label = y_label.to(device)
            y_pred=model(x)
            y_pred_list.extend(y_pred.tolist())
            y_list.extend(y_label.tolist())

        y_pred_list=np.array(y_pred_list)
        y_pred_list=y_pred_list.reshape(y_pred_list.shape[0],1)
        y_list=np.array(y_list)
        y_list=y_list.reshape(y_list.shape[0],1)
    return y_pred_list

In [41]:
epoch = 15
batch_size = 32
lr = 1e-4
input_length = 24
pred_length = 24

train_dataset = dataset(input_length,pred_length,train = 'train')
test_dataset = dataset(input_length,pred_length,train = 'test')


train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,drop_last=False)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True,drop_last=False)

In [42]:
model = AutoFormer(pred_length,2,1,nfeat=4,nembed=8,nhid = 512,nhead = 8)
loss_fn = torch.nn.MSELoss()
optim = torch.optim.Adam(model.parameters(),lr=lr)
schedule = torch.optim.lr_scheduler.CosineAnnealingLR(optim,T_max=len(train_dataloader)*epoch,eta_min=1e-7)

for e in range(epoch):
    print(f'Epoch {e+1}/{epoch}:')
    loss_train = train_per_epoch(train_dataloader)
    report = valid_per_epoch(test_dataloader)
    print('Train loss is {:.5f}'.format(loss_train))
    print('Test loss is {:.5f} - MAPE is {:.5f} - MSE is {:.5f} - RMSE is {:.5f}'.format(report['loss'],report['mape'],report['mse'],report['rmse']))
    print('-----------------------------------------------------')

Epoch 1/15:


100%|██████████| 1/1 [00:00<00:00, 25.65it/s]


Train loss is 2.01741
Test loss is 1.13109 - MAPE is 0.36192 - MSE is 1.13109 - RMSE is 1.06353
-----------------------------------------------------
Epoch 2/15:


100%|██████████| 1/1 [00:00<00:00, 27.98it/s]


Train loss is 1.72692
Test loss is 0.88522 - MAPE is 0.33806 - MSE is 0.88522 - RMSE is 0.94086
-----------------------------------------------------
Epoch 3/15:


...

Train loss is 1.38446
Test loss is 0.61982 - MAPE is 0.26712 - MSE is 0.61982 - RMSE is 0.78728
-----------------------------------------------------
Epoch 14/15:


100%|██████████| 1/1 [00:00<00:00, 29.60it/s]


Train loss is 1.38314
Test loss is 0.61804 - MAPE is 0.26662 - MSE is 0.61804 - RMSE is 0.78616
-----------------------------------------------------
Epoch 15/15:


100%|██████████| 1/1 [00:00<00:00, 29.31it/s]

Train loss is 1.38512
Test loss is 0.61801 - MAPE is 0.26664 - MSE is 0.61801 - RMSE is 0.78614
-----------------------------------------------------
